In [1]:
import pandas as pd
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import cross_validate
from surprise import dump

# Đọc dữ liệu từ file CSV
df_san_pham = pd.read_csv("San_pham.csv")
df_collab = pd.read_csv("collab_df.csv")

# Đổi tên cột để dễ làm việc
df_collab = df_collab.rename(columns={
    "ma_khach_hang": "user_id",
    "ma_san_pham": "item_id",
    "so_sao": "rating"
})

# Chuẩn bị dữ liệu cho Surprise
reader = Reader(rating_scale=(0.5, 5))  # Thang điểm từ 0.5 đến 5 sao
data = Dataset.load_from_df(df_collab[['user_id', 'item_id', 'rating']], reader)

# Khởi tạo mô hình KNNBasic và huấn luyện
algorithm = KNNBasic()
trainset = data.build_full_trainset()
algorithm.fit(trainset)

# Cross-validation để đánh giá mô hình
results = cross_validate(algorithm, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Huấn luyện mô hình trên toàn bộ dữ liệu
algorithm.fit(trainset)

# Lưu mô hình vào file
dump.dump('knn_model.pkl', algo=algorithm)

# Hàm đề xuất sản phẩm
def recommend_products(user_id, top_n=5, threshold=3.5):
    if user_id not in df_collab['user_id'].unique():
        return pd.DataFrame(columns=['ma_san_pham', 'ten_san_pham', 'EstimateScore'])
    
    all_items = df_collab['item_id'].unique()
    rated_items = df_collab[df_collab['user_id'] == user_id]['item_id'].unique()
    items_to_predict = list(set(all_items) - set(rated_items))

    predictions = []
    for item_id in items_to_predict:
        try:
            est_score = algorithm.predict(user_id, item_id).est
            if est_score >= threshold:
                predictions.append((item_id, est_score))
        except Exception as e:
            print(f"Error predicting for user {user_id} and item {item_id}: {e}")
            pass

    if not predictions:
        return pd.DataFrame(columns=['ma_san_pham', 'ten_san_pham', 'EstimateScore'])

    predictions = sorted(predictions, key=lambda x: x[1], reverse=True)[:top_n]
    recommended_items = pd.DataFrame(predictions, columns=['ma_san_pham', 'EstimateScore'])
    recommended_items = recommended_items.merge(df_san_pham, on='ma_san_pham', how='left')

    return recommended_items[['ma_san_pham', 'ten_san_pham', 'gia_ban', 'gia_goc', 'phan_loai', 'mo_ta', 'diem_trung_binh', 'EstimateScore']]

# Ví dụ: Đề xuất sản phẩm cho người dùng có user_id là 1
user_id = 1
recommendations = recommend_products(user_id=user_id, top_n=5)

# In ra danh sách sản phẩm được đề xuất
print(f"Sản phẩm được đề xuất cho người dùng {user_id}:")
print(recommendations)

# Tải lại mô hình đã lưu khi cần thiết
loaded_model = dump.load('knn_model.pkl')[1]

# Giả sử bạn muốn sử dụng mô hình đã tải lại
print("Mô hình đã được tải lại và sẵn sàng sử dụng!")


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.7337  0.7438  0.7831  0.7226  0.7637  0.7494  0.0216  
MAE (testset)     0.3134  0.3224  0.3401  0.2996  0.3302  0.3211  0.0139  
Fit time          0.61    0.54    0.51    0.49    0.50    0.53    0.04    
Test time         0.47    0.50    0.48    0.48    0.47    0.48    0.01    
Computing the msd similarity matrix...
Done computing similarity matrix.
Sản phẩm được đề xuất cho người dùng 1:
   ma_san_pham 